In [2]:
import os


def result_file_name(args):
    file_name = f"results_{args.fold_range}_{args.seed}"
    if len(args.modalities) > 0:
        file_name += "_"
        file_name += "_".join(args.modalities)
    return file_name

class Args:
    def __init__(self, modalities=["clinical", "miRNA", "mRNA", "WSI"]):
        self.epochs = 20
        self.modality_data_path = {'clinical': './preprocess/preprocessed_data/clinical_kidney.csv',
                                    'mRNA': './preprocess/preprocessed_data/mrna_kidney.csv',
                                    'miRNA': './preprocess/preprocessed_data/mirna_kidney.csv',
                                    'WSI': './preprocess/preprocessed_data/UNI2_features/TCGA-Kidney.pt'}
        self.device = "cuda"
        self.modalities = modalities
        self.input_modality_dim = {'clinical':4, 'mRNA':2746, 'miRNA':743 , 'WSI': 1536}
        self.fold_range = 5
        self.fold = 1
        self.modality_fv_len = 128
        self.batch_size = 128
        self.learning_rate = 1e-4
        self.loss_trade_off = 0.3
        self.loss_mode = 'total'
        self.seed = 24
        self.result_path = f"./logs/dgsurv/{result_file_name(self)}"
        print(f"Result Path: {self.result_path}")
        self.log_file_name = "temp.log"
        self.scheduler_patience = 5
        self.num_workers = 4
        self.num_modalities = len(self.modalities)
        self.split_path = "./splits/kidney_splits"
        self.remove_missing = True

args = Args()

Result Path: ./logs/dgsurv/results_5_24_clinical_miRNA_mRNA_WSI


In [3]:
import math
import torch
from torch import nn
import torch.nn.functional as F
from method.GraphLearner import GraphLearner
from method.GraphPooling import GraphPooling
from method.GraphDataUtils import GraphInputProcessor


class GraphModel(nn.Module):
    def __init__(self, n_views, n_in_feats, encoder_model,
                 gnn_arch='gcn',
                 pool_ratio=.1, sparse_threshold=0.1,
                 device='cuda'):
        super(GraphModel, self).__init__()
        self.encoder_model = encoder_model

        n_edge_types = int(n_views**2)
        self.graph_input_processor = GraphInputProcessor(
            n_edge_types=n_edge_types, device=device)

        self.graph_learner = GraphLearner(
            n_in_feats=n_in_feats+n_views,
            n_out_feats=100,
            threshold=sparse_threshold,
            n_heads=1,
            device=device
        )

        n_nodes = max(math.ceil(n_views*pool_ratio), 1)
        self.n_nodes = n_nodes

        n_feats = n_in_feats
        self.pool = GraphPooling(
            n_in_feats, n_nodes,
            n_feats, n_edge_types,
            gnn_arch=gnn_arch, pool=True
        )

        self.output_dim = n_feats
        self.hazard_layer1 = nn.Linear(self.output_dim, 1)

        self.label_layer1 = nn.Linear(self.output_dim, 2)
			
    def forward(self, x_modality, mask):
        representation = self.encoder_model(x_modality)
        missing_rep = []
        for i, rep in enumerate(representation):
            index = torch.ones((rep.shape[0]), dtype=int) * i 
            index = index.to(rep.device)
            modality_mask = mask[:, i].reshape(-1, 1)
            rep =  modality_mask * rep
            missing_rep.append(rep)
        
        representation_dict = {} #used for self supervised loss
        for i, data in enumerate(missing_rep):
            representation_dict[i] = data
                     
        in_graph = self.graph_input_processor(missing_rep)
        learned_graph, _ = self.graph_learner(in_graph)
        latent_graph, assignment_mat = self.pool(learned_graph)
        final_representation = latent_graph[0].squeeze(1)
        hazard = self.hazard_layer1(final_representation)
        score = F.log_softmax(self.label_layer1(final_representation), dim=1)
        return {'hazard':hazard, 'score':score}, representation_dict, learned_graph[1], assignment_mat

In [4]:
import numpy as np
from lifelines.utils import concordance_index


def print_log(msg, logger):
    print(msg)
    logger.info(msg)

def evaluation(model, dataloader, loss_fucnt, device, graph_save_path=None):
    running_loss = []
    running_graph = []
    running_ass_mat = []
    running_sample_time = torch.FloatTensor().to(device)
    running_sample_event = torch.LongTensor().to(device)
    running_hazard = torch.FloatTensor().to(device)
    
    model.eval()
    with torch.no_grad():
        for x_modal, data_label, mask in dataloader:
            for modality in x_modal:
                x_modal[modality] = x_modal[modality].to(device)
            event = data_label[:, 0].to(device)
            time = data_label[:, 1].to(device)
            mask = mask.to(device)
            hazard, representation, graph, assignment_mat = model(x_modal, mask)
            if graph_save_path:
                running_graph.extend(graph.detach().cpu().numpy())
                running_ass_mat.extend(assignment_mat.detach().cpu().numpy())
            loss = loss_fucnt(representation, list(representation.keys()), hazard, event, time)
            hazard = hazard['hazard']
            
            running_loss.append(loss.item())
            running_sample_time = torch.cat((running_sample_time, time.data.float()))
            running_sample_event = torch.cat((running_sample_event, event.long().data))
            running_hazard = torch.cat((running_hazard, hazard.detach()))
            
        if graph_save_path:
            np.save(graph_save_path + "graph.npy", np.array(running_graph))
            np.save(graph_save_path + "assignment_mat.npy", np.array(running_ass_mat)) 
    eval_loss = torch.mean(torch.tensor(running_loss))
    eval_c_index = concordance_index(running_sample_time.cpu().numpy(), -running_hazard.cpu().numpy(), running_sample_event.cpu().numpy())
    return eval_loss, eval_c_index

In [5]:
import copy
import random
import logging 
from utils.loss import Loss
from torch.optim import Adam
from torch.utils.data import DataLoader
from utils.dataset import MultiModalDataset
from utils.encoder import EncoderModel
    

def save_model(model, path):
    torch.save(model.state_dict(), path)

def set_seed(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True
        
def train_eval_model(args):
    set_seed(args.seed)
    os.makedirs(args.result_path, exist_ok=True)

    logging.basicConfig(format='%(message)s')
    for k in range(args.fold_range):
        args.fold = k
        args.log_file_name = f"result_fold{args.fold}.log"
        args.log_file_name = os.path.join(args.result_path, args.log_file_name)
        if os.path.exists(args.log_file_name):
            os.remove(args.log_file_name)
            
        handler = logging.FileHandler(args.log_file_name, mode='w')
        logger=logging.getLogger() 
        logger.setLevel(logging.DEBUG) 
        logger.addHandler(handler)
        train_dataset = MultiModalDataset(args, 'train', args.modalities, args.modality_data_path, remove_missing=args.remove_missing)
        train_dataloader = DataLoader(train_dataset, batch_size=args.batch_size, shuffle=True, num_workers=args.num_workers)
        test_dataset = MultiModalDataset(args, 'test', args.modalities, args.modality_data_path)
        test_dataloader = DataLoader(test_dataset, batch_size=args.batch_size, shuffle=False, num_workers=args.num_workers)

        device = torch.device(args.device)
        encoder_model = EncoderModel(args.modalities, args.modality_fv_len, args.input_modality_dim)
        model = GraphModel(args.num_modalities, args.modality_fv_len, encoder_model, device=args.device)
        optimizer = Adam(model.parameters(), lr=args.learning_rate)
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau( 
            optimizer=optimizer, mode='max', factor=0.1,
            patience = args.scheduler_patience, verbose=True, threshold=1e-3,
            threshold_mode='rel', cooldown=0, min_lr=0, eps=1e-8)
        model.to(device)
        
        loss_fucnt = Loss(trade_off=args.loss_trade_off, mode=args.loss_mode)

        for epoch in range(args.epochs):
            running_loss = []
            running_sample_time = torch.FloatTensor().to(device)
            running_sample_event = torch.LongTensor().to(device)
            running_hazard = torch.FloatTensor().to(device)
            
            model.train()
            for x_modal, data_label, mask in train_dataloader:
                for modality in x_modal:
                    x_modal[modality] = x_modal[modality].to(device)
                event = data_label[:, 0].to(device)
                time = data_label[:, 1].to(device)
                mask = mask.to(device)
                hazard, representation, _, _ = model(x_modal, mask)
                loss = loss_fucnt(representation, list(representation.keys()), hazard, event, time)
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                hazard = hazard['hazard']
                
                running_loss.append(loss.item())
                running_sample_time = torch.cat((running_sample_time, time.data.float()))
                running_sample_event = torch.cat((running_sample_event, event.long().data))
                running_hazard = torch.cat((running_hazard, hazard.detach()))
                
            train_loss = torch.mean(torch.tensor(running_loss))
            train_c_index = concordance_index(running_sample_time.cpu().numpy(), -running_hazard.cpu().numpy(), running_sample_event.cpu().numpy())
            print_log(f"Epoch {epoch}/{args.epochs}: Training Loss: {train_loss}, C-Index: {train_c_index}", logger)
            scheduler.step(train_c_index)
            if scheduler.patience == scheduler.num_bad_epochs: 
                print_log("Reducing the learning rate", logger)
            best_model = copy.deepcopy(model.state_dict())
        print_log("*" * 100, logger)
        
        model.load_state_dict(best_model)
        model_save_path = os.path.join(args.result_path, f"model_fold{args.fold}.pt")
        save_model(model, model_save_path)
        graph_save_path = os.path.join(args.result_path, f"vis_{args.fold}")
        train_loss, train_c_index = evaluation(model, train_dataloader, loss_fucnt, device, graph_save_path + "_train_")
        print_log(f"Final Train Loss: {train_loss}, C-Index: {train_c_index}", logger)
        test_loss, test_c_index = evaluation(model, test_dataloader, loss_fucnt, device, graph_save_path + "_test_")
        print_log(f"Test Loss: {test_loss}, C-Index: {test_c_index}", logger)
        
        logger.removeHandler(handler)

In [6]:
import numpy as np


all_modalities = ["clinical", "miRNA", "mRNA", "WSI"]
all_modalities = np.array(all_modalities)
i = 2 ** len(all_modalities) - 1
selection = F"{i:0{len(all_modalities)}b}"
modality_select = all_modalities[np.array(list(selection)) == '1']
args = Args(modality_select)
train_eval_model(args)

Result Path: ./logs/dgsurv/results_5_24_clinical_miRNA_mRNA_WSI


Epoch 0/20: Training Loss: 4.296722412109375, C-Index: 0.5663001642666156


Epoch 0/20: Training Loss: 4.296722412109375, C-Index: 0.5663001642666156


Epoch 1/20: Training Loss: 4.106428623199463, C-Index: 0.73060275056654


Epoch 1/20: Training Loss: 4.106428623199463, C-Index: 0.73060275056654


...

Epoch 19/20: Training Loss: 2.887754440307617, C-Index: 0.917814050793033
****************************************************************************************************


Final Train Loss: 2.8697750568389893, C-Index: 0.9281283448477182


Final Train Loss: 2.8697750568389893, C-Index: 0.9281283448477182


Test Loss: 3.477527618408203, C-Index: 0.7878732030962035


Test Loss: 3.477527618408203, C-Index: 0.7878732030962035
